In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pde
import os


# Boundary conditions test

In [22]:
class CustomPDE(pde.PDEBase):
    def __init__(self, boundary_mask):
        self.boundary_mask = boundary_mask

    def evolution_rate(self, state, t=0):
        """ Custom PDE evolution with modified diffusion rate """
        laplacian = state.laplace(bc={"value": 0})  # zero Dirichlet boundary condition
        rate = laplacian.copy()  # Default rate inside the grid
        
        # Reduce the rate of diffusion at grid to a quarter
        rate.data[self.boundary_mask] *= 0.25
        
        return rate


In [23]:
# generate grid
grid_size = [64, 64]
grid = pde.UnitGrid(grid_size)  
state = pde.ScalarField(grid, data=0)

 # insert peaks in the centers
state.insert([10.5, 10.5], 1)
state.insert([32, 10.5], 1)
state.insert([53.3, 10.5], 1)
state.insert([10.5, 32], 1)
state.insert([32, 32], 1) 
state.insert([53.3, 32], 1)
state.insert([10.5, 53.3], 1)
state.insert([32, 53.3], 1)
state.insert([53.3, 53.3], 1)

# Define the mask for grid lines with thickness 1
x, y = grid.cell_coords[..., 0], grid.cell_coords[..., 1]
section_size = grid_size[0] // 3
boundary_mask = (
    ((x % section_size <= 2) | (x % section_size >= section_size - 2)) |
    ((y % section_size <= 2) | (y % section_size >= section_size - 2))
)

eq = CustomPDE(boundary_mask)
storage = pde.MemoryStorage()
result = eq.solve(state, t_range=30, dt=0.1, tracker=storage.tracker(0.1))

In [24]:
# Plot calculated results
# WARNING: This will take a long time to run, uncomment with care
pde.movie(storage, filename="output.mp4")

100%|██████████| 300/300 [00:06<00:00, 44.50it/s]


# JIT test

In [25]:
from pde.tools.numba import jit

In [26]:

class KuramotoSivashinskyPDE(pde.PDEBase):

    def __init__(self, diffusivity=1, bc="auto_periodic_neumann", bc_laplace="auto_periodic_neumann"):
        """ initialize the class with a diffusivity and boundary conditions
        for the actual field and its second derivative """
        self.diffusivity = diffusivity
        self.bc = bc
        self.bc_laplace = bc_laplace


    def evolution_rate(self, state, t=0):
        """ numpy implementation of the evolution equation """
        state_lapacian = state.laplace(bc=self.bc)
        state_gradient = state.gradient(bc="auto_periodic_neumann")
        return (- state_lapacian.laplace(bc=self.bc_laplace)
                - state_lapacian
                - 0.5 * self.diffusivity * (state_gradient @ state_gradient))


    def _make_pde_rhs_numba(self, state):
        """ the numba-accelerated evolution equation """
        # make attributes locally available
        diffusivity = self.diffusivity

        # create operators
        laplace_u = state.grid.make_operator("laplace", bc=self.bc)
        gradient_u = state.grid.make_operator("gradient", bc=self.bc)
        laplace2_u = state.grid.make_operator("laplace", bc=self.bc_laplace)
        dot = pde.VectorField(state.grid).make_dot_operator()
        dim = state.grid.dim

        @jit
        def pde_rhs(state_data, t=0):
            """ compiled helper function evaluating right hand side """
            state_lapacian = laplace_u(state_data)
            state_grad = gradient_u(state_data)
            result = - laplace2_u(state_lapacian) - state_lapacian

            for i in range(state_data.size):
                for j in range(dim):
                    result.flat[i] -= diffusivity / 2 * state_grad[j].flat[i]**2

            return result

        return pde_rhs

In [27]:
grid_size = [64, 64]
grid = pde.UnitGrid(grid_size, periodic=True)
state = pde.ScalarField(grid, data=np.random.rand(*grid.shape))

eq = KuramotoSivashinskyPDE(diffusivity=0.1)
storage = pde.MemoryStorage()
result = eq.solve(state, t_range=33, dt=0.001, tracker=storage.tracker(0.1))

In [28]:
# Plot calculated results
# WARNING: This will take a long time to run, uncomment with care

# pde.movie(storage, filename="output2.mp4", plot_args={"vmin": -30, "vmax": 30})

# 3D test

In [29]:
# generate grid
grid_size = [64, 64, 64]
grid = pde.UnitGrid(grid_size)  
state = pde.ScalarField(grid, data=0)
state.insert([32, 32, 32], 1)  # insert peak 

# Define the mask for grid lines with thickness of 5 in 3D
x, y, z = grid.cell_coords[..., 0], grid.cell_coords[..., 1], grid.cell_coords[..., 2]
section_size = grid_size[0] // 3
boundary_mask = (
    ((x % section_size <= 2) | (x % section_size >= section_size - 2)) |
    ((y % section_size <= 2) | (y % section_size >= section_size - 2)) |
    ((z % section_size <= 2) | (z % section_size >= section_size - 2))
)

eq = CustomPDE(boundary_mask)
storage = pde.MemoryStorage()
result = eq.solve(state, t_range=30, dt=0.1, tracker=storage.tracker(0.1))


In [30]:
# Plot calculated results in 3D
# WARNING: This will take even longer time to run, uncomment with care

# slice at depth 32
# print(storage[0].data[32].shape)
# print(storage[0].data[32])
# print(storage[0].data[:,:,32])
# print(storage[0])

# @jit
def get_slice(storage, z=32):
    # Transform the 3D storage data to 2D slice at depth z
    grid_size = [64, 64]
    grid = pde.UnitGrid(grid_size)  
    
    new_data = []
    for time in range(len(storage)):
        data=storage[time].data[:,:,z]
        new_data.append(data)
    new_data = np.array(new_data)
    field_obj = pde.ScalarField(grid, data=new_data[0])
    res = pde.storage.memory.MemoryStorage(times=list(range(len(storage))), data=new_data, field_obj=field_obj)
    print("res",res[0])
        
    return res


new_storage = get_slice(storage, 28)
pde.movie(new_storage, filename="output3.mp4", plot_args={"vmin": 0, "vmax": 0.01,})

res ScalarField(grid=UnitGrid(shape=(64, 64), periodic=[False, False]), data=Array(64, 64))


100%|██████████| 300/300 [00:06<00:00, 48.70it/s]
